### Yolov11 Training Pipeline

##### Imports

In [ ]:
import os
import gc
import re
import cv2
import random
import shutil
import torch
import warnings
import torchvision
from datetime import datetime
import matplotlib.pyplot as plt

2.7.0.dev20250127+rocm6.3
0.22.0.dev20250128+rocm6.3
True


##### Package Versions

In [ ]:
print("Torch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("Is torch available?",torch.cuda.is_available())

### Disable Warnings

In [ ]:
warnings.filterwarnings("ignore")

### Setting Training Device

In [ ]:
# Defining Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Printing Necessary CUDA Info
if str(device) == "cuda":
    print(f"Using GPU {torch.cuda.get_device_properties(device)}")
    print("# of CUDA devices available:", torch.cuda.device_count())
else:
    print("Using CPU")

### Emptying the GPU Cache (if necessary)

In [ ]:
# Cleaning out the device cache
def empty_cache() -> None:
    gc.collect()
    torch.cuda.empty_cache()

# Displaying the free memory
def print_free_memory():
    free, total = torch.cuda.mem_get_info(device)
    print(f"Percent of free memory: {round(free/total *100,2)}")

# Running GPU info related functions
empty_cache()
print_free_memory()

### Memory Summary

In [ ]:
# Memory Summary Function
def memory_summary() -> None:
    print(torch.cuda.memory_summary())

# Running memory summary funciton
memory_summary()

### Preparing GPU (if necessary)


In [ ]:
# For AMD GPU - 7800xt
device_name = torch.cuda.get_device_name(0)
if "AMD" in device_name or "Radeon" in device_name:
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = "11.0.0"

print(f"GPU {torch.cuda.get_device_properties(device).name} is now setup")

#### Google Colab Initialization -> Later delete once running on THI servers

In [ ]:
# Additional Package Installations
from google.colab import drive
!pip install ultralytics

# Image Zip Directory
zip_name= "2.5k_background.zip"

# Mounting Google Drive and setting paths
drive.mount('/content/drive', force_remount=True)
zip_path = f"/content/drive/MyDrive/Valomalo/{zip_name}"
unzip_path = "/content"
shutil.unpack_archive(zip_path, unzip_path)

# Directory Paths
dataset_path = f'/content/{zip_name.replace(".zip", "")}/data.yaml'
save_dir = '/content/drive/My Drive/Valomalo'

ModuleNotFoundError: No module named 'google'

##### Local Setting Paths for Data, Base Model, and Output Directory


In [ ]:
# Old Static Dataset Paths -> Can't use on THI server
#dataset_path = r"C:\Users\Adminero\Documents\Valomalo\final dataset_nobackground\cutouts1024\data.yaml" #.yaml file path
#save_dir = r"C:\Users\Adminero\Documents\Valomalo\Saved Runs"

# Directory Paths
inference_dataset_name = "coco8" # Change this once we have the necessary data

# Flexible Data Paths (needed for THI server)
current_directory = os.getcwd()
path_to_base_directory = re.search(rf"(.*?){"Weird-Stuff-In-Traffic"}", current_directory).group(1)
inference_yaml_data_path = f"Weird-Stuff-In-Traffic/Data/yolo/{inference_dataset_name}/coco8.yaml" # Change this once we have the necessary data
complete_inference_data_path = path_to_base_directory + inference_yaml_data_path

# Model Paths
model_name = "yolo11n.pt"
simple_model_name = model_name.split(".")[0]

##### Training - Loading Model

In [ ]:
from ultralytics import YOLO

model = YOLO(model_name)

##### Training - Training Configuration and Output Paths

In [ ]:
# Model Training Configurations
image_size = 620
batch_size = 64

# Output Folder Name
output_folder_name = f'{datetime.now().strftime("%Y-%m-%d_%H-%M")}_{simple_model_name}_{training_dataset_name}_{image_size}cuts_{epochs}epoch'

# Results Path
training_results_path = f"Weird-Stuff-In-Traffic/Results/Segmentation-Detection/yolo/"
complete_training_results_path = path_to_base_directory + training_results_path + output_folder_name

# Inference

In [ ]:
model_path = f'{zip_path.replace(".zip", "")}_v9n/weights/best.pt'
model = YOLO(model_path)

video_path = "prediction_test.mp4"

# Directory to save the annotated video
save_dir = '/content/drive/My Drive/Valomalo/'
experiment = "video_inference_v9"
# Run inference on the video with stream=True (to avoid memory crashes)
results = model.predict(source=video_path,
                        stream=True,
                        imgsz= 640,
                        conf=0.4,
                        max_det=9,
                        show_labels = False,
                        show_conf = False,
                        save=True,
                        project=save_dir,
                        name=experiment)

# Process the results
for r in results:
    boxes = r.boxes  # Boxes object for bbox outputs
    masks = r.masks  # Masks object for segment masks outputs
    probs = r.probs  # Class probabilities for classification outputs

print(f"Annotated video saved to {save_dir}/{experiment}")

# Hyperparameter Tuning

## Builtin ray tune (Hyperband)

In [ ]:
from ultralytics import YOLO
from ray import tune

model_name = "yolo11n.pt"
model = YOLO(model_name)
hypp_space = {}

result_grid = model.tune(
    device='cuda:0',
    data=dataset_path,
    iterations=3,
    grace_period = 2,
    name="tune_exp_1", 
    #space = hypp_space,
    #resume=True,
    imgsz=1024,
    epochs=2,
    batch=24,
    project=save_dir,
    use_ray=True)


(_tune pid=26916) New https://pypi.org/project/ultralytics/8.3.112 available 😃 Update with 'pip install -U ultralytics'
(_tune pid=26916) Ultralytics 8.3.111 🚀 Python-3.12.10 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
(_tune pid=26916) engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=C:\Users\Adminero\Documents\Valomalo\final dataset_nobackground\cutouts1024\data.yaml, epochs=2, time=None, patience=100, batch=24, imgsz=1024, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=C:\Users\Adminero\Documents\Valomalo\Saved Runs, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source

  0%|          | 0.00/5.35M [00:00<?, ?B/s]
  7%|▋         | 384k/5.35M [00:00<00:01, 3.10MB/s]
100%|██████████| 5.35M/5.35M [00:01<00:00, 5.14MB/s]
100%|██████████| 5.35M/5.35M [00:01<00:00, 4.95MB/s]
100%|██████████| 5.35M/5.35M [00:01<00:00, 3.18MB/s]


(_tune pid=26916) AMP: checks passed ✅


train: Scanning C:\Users\Adminero\Documents\Valomalo\final dataset_nobackground\cutouts1024\train\labels.cache... 2471 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2471/2471 [00:00<?, ?it/s]


(_tune pid=26916) train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 385.3±55.1 MB/s, size: 173.3 KB)


  0%|          | 0.00/5.35M [00:00<?, ?B/s] [repeated 2x across cluster]
 84%|████████▍ | 4.50M/5.35M [00:01<00:00, 4.48MB/s] [repeated 27x across cluster]
val: Scanning C:\Users\Adminero\Documents\Valomalo\final dataset_nobackground\cutouts1024\val\labels.cache... 308 images, 0 backgrounds, 0 corrupt: 100%|██████████| 308/308 [00:00<?, ?it/s] [repeated 3x across cluster]


(_tune pid=27636) Overriding model.yaml nc=80 with nc=2 [repeated 2x across cluster]
(_tune pid=27636)                    from  n    params  module                                       arguments                      [repeated 2x across cluster]
(_tune pid=27636)  20                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]               [repeated 14x across cluster]
(_tune pid=27636)   4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]      [repeated 4x across cluster]
(_tune pid=27636)  22                  -1  1    378880  ultralytics.nn.modules.block.C3k2            [384, 256, 1, True]            [repeated 6x across cluster]
(_tune pid=27636)   9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                  [repeated 2x across cluster]
(_tune pid=27636)  14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2

2025-04-21 17:59:31,882	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2025-04-21 17:59:31,886	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/Adminero/Documents/Valomalo/Saved Runs/tune_exp_18' in 0.0033s.
  0%|          | 0/103 [00:00<?, ?it/s]
val: Scanning C:\Users\Adminero\Documents\Valomalo\final dataset_nobackground\cutouts1024\val\labels.cache... 308 images, 0 backgrounds, 0 corrupt: 100%|██████████| 308/308 [00:00<?, ?it/s] [repeated 2x across cluster]


(_tune pid=27636) optimizer: 'optimizer=auto' found, ignoring 'lr0=0.016354728253908123' and 'momentum=0.7360041068128451' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
(_tune pid=27636) optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0008806517954492481), 87 bias(decay=0.0)
(_tune pid=27636) Image sizes 1024 train, 1024 val
(_tune pid=27636) Using 8 dataloader workers
(_tune pid=27636) Logging results to C:\Users\Adminero\Documents\Valomalo\Saved Runs\train13
(_tune pid=27636) Starting training for 2 epochs...
(_tune pid=27636) 
(_tune pid=27636)       Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
(_tune pid=27132) Plotting labels to C:\Users\Adminero\Documents\Valomalo\Saved Runs\train14\labels.jpg...  [repeated 2x across cluster]
(_tune pid=27132) 
